In [2]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt') # one time execution
import re

ModuleNotFoundError: No module named 'pandas'

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract
import pytesseract
import shutil
import os
import random
import io
!pip install -U pillow
try:
    from PIL import Image
except ImportError:
    import Image

In [ ]:
from google.colab import files
uploaded = files.upload()
filename = next(iter(uploaded))
ext = filename.split(".")[-1]
ext

In [ ]:
#df = pd.read_csv(io.StringIO(uploaded[filename].decode("utf-8")), sep='\t')

In [ ]:
if (ext == 'txt') :
  df = pd.read_csv(io.StringIO(uploaded[filename].decode("utf-8")), sep='\t')
elif (ext == 'png' or 'jpg'):
  extractedInformation = pytesseract.image_to_string(Image.open(filename))
  df = extractedInformation
  import re
  #re.sub(r'\s+', '', df)
  df = " ".join(re.split("\s+", df, flags=re.UNICODE))
  df=df.split('.')
else:
  print("error")

In [ ]:
sentences = []
for s in df:
  sentences.append(sent_tokenize(s))  

In [ ]:
sentences = [y for x in sentences for y in x]

In [ ]:
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

clean_sentences = [s.lower() for s in clean_sentences]

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [ ]:
def remove_stopwords(sen):
  sen_new = " ".join([i for i in sen if i not in stop_words])
  return sen_new

In [ ]:
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [ ]:
#! wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
#!unzip glove*.zip

In [ ]:
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [ ]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [ ]:
len(sentence_vectors)

In [ ]:
sim_mat = np.zeros([len(sentences), len(sentences)])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [ ]:
import networkx as nx
nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [ ]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [ ]:
sn = 3
summ=[ ]
for i in range(sn):
  print(ranked_sentences[i][1])
for i in range(sn):
  summ.append("".join(ranked_sentences[i][1]))

In [ ]:
s = ' '.join(map(str, summ))

In [ ]:
!pip install gtts
from gtts import gTTS 
from IPython.display import Audio 
for i in range(sn):
  tts = gTTS(s) 
tts.save('1.wav') 
sound_file = '1.wav'
Audio(sound_file, autoplay=True) 

In [ ]:
!pip install py-rouge

In [3]:
import rouge
import nltk
nltk.download('punkt')
def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)


for aggregator in ['Avg', 'Best', 'Individual']:
    print('Evaluation with {}'.format(aggregator))
    apply_avg = aggregator == 'Avg'
    apply_best = aggregator == 'Best'

    evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                           max_n=4,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=apply_avg,
                           apply_best=apply_best,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)

    references_1 ="TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn.For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 2003 performance, while revenues grew 6.4% to $42.09bn.Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues.Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.For 2005, TimeWarner is projecting operating earnings growth of around 5%, and also expects higher revenue and wider profit margins.It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters.Time Warner's fourth quarter profits were slightly better than analysts' expectations."
    hypothesis_1 ="Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (£600m) for the three months to December, from $639m year-earlier.The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.Time Warner said on Friday that it now owns 8% of search-engine Google."
    
    references_2 ="The dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise.China's currency remains pegged to the dollar and the US currency's sharp falls in recent months have therefore made Chinese export prices highly competitive.Market concerns about the deficit has hit the greenback in recent months.I think the chairmans taking a much more sanguine view on the current account deficit than hes taken for some time said Robert Sinche, head of currency strategy at Bank of America in New York.The recent falls have partly been the result of big budget deficits, as well as the US's yawning current account gap, both of which need to be funded by the buying of US bonds and assets by foreign firms and governments.He's taking a longer-term view, laying out a set of conditions under which the current account deficit can improve this year and next."
    hypothesis_2 = "Dollar gains on Greenspan speech.The dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise.And Alan Greenspan highlighted the US government's willingness to curb spending and rising household savings as factors which may help to reduce it. In the meantime, the US Federal Reserve's decision on 2 February to boost interest rates by a quarter of a point - the sixth such move in as many months - has opened up a differential with European rates. The recent falls have partly been the result of big budget deficits, as well as the US's yawning current account gap, both of which need to be funded by the buying of US bonds and assets by foreign firms and governments."

    references_3 ="Yukos' owner Menatep Group says it will ask Rosneft to repay a loan that Yugansk had secured on its assets.State-owned Rosneft bought the Yugansk unit for $9.3bn in a sale forced by Russia to part settle a $27.5bn tax claim against Yukos.The sale went ahead in December and Yugansk was sold to a little-known shell company which in turn was bought by Rosneft.But the company has said it intends to take action against Menatep to recover some of the tax claims and debts owed by Yugansk.The pledged assets are with Rosneft, so it will have to pay real money to the creditors to avoid seizure of Yugansk assets,said Moscow-based US lawyer Jamie Firestone, who is not connected to the case."
    hypothesis_3 = "The pledged assets are with Rosneft, so it will have to pay real money to the creditors to avoid seizure of Yugansk assets, said Moscow-based US lawyer Jamie Firestone, who is not connected to the case. Yukos unit buyer faces loan claim.The owners of embattled Russian oil giant Yukos are to ask the buyer of its former production unit to pay back a $900m (£479m) loan.State-owned Rosneft bought the Yugansk unit for $9.3bn in a sale forced by Russia to part settle a $27.5bn tax claim against Yukos. Legal experts said Rosneft's purchase of Yugansk would include such obligations."

    references_4 ="Rod Eddington, BA's chief executive, said the results were respectable in a third quarter when fuel costs rose by £106m or 47.3%.To help offset the increased price of aviation fuel, BA last year introduced a fuel surcharge for passengers.BA had previously forecast a 2% to 3% rise in full-year revenue.It is quite good on the revenue side and it shows the impact of fuel surcharges and a positive cargo development, however, operating margins down and cost impact of fuel are very strong,he said.Yet aviation analyst Mike Powell of Dresdner Kleinwort Wasserstein says BA's estimated annual surcharge revenues - £160m - will still be way short of its additional fuel costs - a predicted extra £250m.For the year to March 2005, the total revenue outlook is slightly better than previous guidance with a 3% to 3.5% improvement anticipated,BA chairman Martin Broughton said.Looking ahead to its full year results to March 2005, BA warned that yields - average revenues per passenger - were expected to decline as it continues to lower prices in the face of competition from low-cost carriers.BA's profits were still better than market expectation of £59m, and it expects a rise in full-year revenues."
    hypothesis_4 = "BA's profits were still better than market expectation of £59m, and it expects a rise in full-year revenues.To help offset the increased price of aviation fuel, BA last year introduced a fuel surcharge for passengers.In October, it increased this from £6 to £10 one-way for all long-haul flights, while the short-haul surcharge was raised from £2.50 to £4 a leg. High fuel prices hit BA's profits.British Airways has blamed high fuel prices for a 40% drop in profits.Reporting its results for the three months to 31 December 2004, the airline made a pre-tax profit of £75m ($141m) compared with £125m a year earlier. Looking ahead to its full year results to March 2005, BA warned that yields - average revenues per passenger - were expected to decline as it continues to lower prices in the face of competition from low-cost carriers."

    references_5 ="Pernod has reduced the debt it took on to fund the Seagram purchase to just 1.8bn euros, while Allied has improved the performance of its fast-food chains.Shares in UK drinks and food firm Allied Domecq have risen on speculation that it could be the target of a takeover by France's Pernod Ricard.Pernod said it was seeking acquisitions but refused to comment on specifics.In terms of market value, Pernod - at 7.5bn euros ($9.7bn) - is about 9% smaller than Allied Domecq, which has a capitalisation of £5.7bn ($10.7bn; 8.2bn euros).Allied Domecq shares in London rose 4% by 1200 GMT, while Pernod shares in Paris slipped 1.2%."
    hypothesis_5 = "Shares in UK drinks and food firm Allied Domecq have risen on speculation that it could be the target of a takeover by France's Pernod Ricard..Reports in the Wall Street Journal and the Financial Times suggested that the French spirits firm is considering a bid, but has yet to contact its target. Last year Pernod tried to buy Glenmorangie, one of Scotland's premier whisky firms, but lost out to luxury goods firm LVMH. Pernod said it was seeking acquisitions but refused to comment on specifics.Pernod's last major purchase was a third of US giant Seagram in 2000, the move which propelled it into the global top three of drinks firms."

    all_hypothesis = [hypothesis_1, hypothesis_2]
    all_references = [references_1, references_2]

    scores = evaluator.get_scores(all_hypothesis, all_references)

    for metric, results in sorted(scores.items(), key=lambda x: x[0]):
        if not apply_avg and not apply_best: # value is a type of list as we evaluate each summary vs each reference
            for hypothesis_id, results_per_ref in enumerate(results):
                nb_references = len(results_per_ref['p'])
                for reference_id in range(nb_references):
                    print('\tHypothesis #{} & Reference #{}: '.format(hypothesis_id, reference_id))
                    print('\t' + prepare_results(results_per_ref['p'][reference_id], results_per_ref['r'][reference_id], results_per_ref['f'][reference_id]))
            print()
        else:
            print(prepare_results(results['p'], results['r'], results['f']))
    print()

ModuleNotFoundError: No module named 'nltk'